# LMDeploy环境搭建

## LMDeploy介绍
书生浦语首页网址：
[书生浦语](https://internlm.intern-ai.org.cn/)

中文使用说明网址：
[中文使用说](https://lmdeploy.readthedocs.io/zh-cn/latest/)

## 安装

### 硬件要求

![硬件要求](Image/2025-03-26-20-52-46.png)

### 官网安装方法

![官网安装方法](Image/2025-03-26-20-59-29.png)

执行一下命令进行安装

conda create -n lmdeploy python=3.8 -y
conda activate lmdeploy
pip install lmdeploy

## 开启LLM 模型服务
开启方法：
lmdeploy serve api_server +模型路径

例如：

lmdeploy serve api_server internlm/internlm2_5-7b-chat

开启千问模型的服务

lmdeploy serve api_server /root/autodl-tmp/model/Qwen/Qwen2.5-0.5B-Instruct

如果报错则安装缺少的东西

![](Image/2025-03-26-22-11-23.png)

![](Image/2025-03-26-22-23-36.png)

端口号默认：23333，代码需要修改端口号和模型路径

![](Image/2025-03-26-22-12-37.png)

![](Image/2025-03-26-22-24-03.png)

In [ ]:
#多轮对话
from openai import OpenAI

#定义多轮对话方法
def run_chat_session():
    #初始化客户端
    client = OpenAI(base_url="http://localhost:23333/v1/",api_key="suibianxie")
    #初始化对话历史
    chat_history = []
    #启动对话循环
    while True:
        #获取用户输入
        user_input = input("用户：")
        if user_input.lower() == "exit":
            print("退出对话。")
            break
        #更新对话历史(添加用户输入)
        chat_history.append({"role":"user","content":user_input})
        #调用模型回答
        try:
            chat_complition = client.chat.completions.create(messages=chat_history,model="/root/autodl-tmp/model/Qwen/Qwen2.5-0.5B-Instruct")
            #获取最新回答
            model_response = chat_complition.choices[0]
            print("AI:",model_response.message.content)
            #更新对话历史（添加AI模型的回复）
            chat_history.append({"role":"assistant","content":model_response.message.content})
        except Exception as e:
            print("发生错误：",e)
            break
if __name__ == '__main__':
    run_chat_session()

# 1.LoRA微调的基本原理

## 基本概念


 LoRA（Low-Rank Adaptation）是一种用于大模型微调的技术，通过引入低秩矩阵来减少微调时的参数量。在预训练的模型中，LoRA通过添加两个小矩阵B和A来近似原始的大矩阵ΔW，从而减少需要更新的参数数量。具体来说，LoRA通过将全参微调的增量参数矩阵ΔW表示为两个参数量更小的矩阵B和A的低秩近似来实现：
 [ W_0 + \Delta W = W_0 + BA ]
 其中，B和A的秩远小于原始矩阵的秩，从而大大减少了需要更新的参数数量。

## 思想

预训练模型中存在一个极小的内在维度，这个内在维度是发挥核心作用的地方。在继续训练的过程中，权重的更新依然也有如此特点，即也存在一个内在维度(内在秩)
 • 权重更新:W=W+^W
 • 因此，可以通过矩阵分解的方式，将原本要更新的大的矩阵变为两个小的矩阵
 • 权重更新:W=W+^W=W+BA
 • 具体做法，即在矩阵计算中增加一个旁系分支，旁系分支由两个低秩矩阵A和B组成


## 原理

• 训练时，输入分别与原始权重和两个低秩矩阵进行计算，共同得到最终结果，优化则仅优化A和B
• 训练完成后，可以将两个低秩矩阵与原始模型中的权重进行合并，合并后的模型与原始模型无异


![](Image/2025-03-26-23-27-22.png)

# 2.LLaMA-Factory介绍

[LLaMA-Factory Github介绍](https://github.com/hiyouga/LLaMA-Factory/blob/main/README_zh.md)

[LLaMA-Factory官方保姆级教程](https://blog.csdn.net/python12345678_/article/details/140346926)

## 项目特色
多种模型：LLaMA、LLaVA、Mistral、Mixtral-MoE、Qwen、Qwen2-VL、DeepSeek、Yi、Gemma、ChatGLM、Phi 等等。
集成方法：（增量）预训练、（多模态）指令监督微调、奖励模型训练、PPO 训练、DPO 训练、KTO 训练、ORPO 训练等等。
多种精度：16 比特全参数微调、冻结微调、LoRA 微调和基于 AQLM/AWQ/GPTQ/LLM.int8/HQQ/EETQ 的 2/3/4/5/6/8 比特 QLoRA 微调。
先进算法：GaLore、BAdam、APOLLO、Adam-mini、DoRA、LongLoRA、LLaMA Pro、Mixture-of-Depths、LoRA+、LoftQ 和 PiSSA。
实用技巧：FlashAttention-2、Unsloth、Liger Kernel、RoPE scaling、NEFTune 和 rsLoRA。
广泛任务：多轮对话、工具调用、图像理解、视觉定位、视频识别和语音理解等等。
实验监控：LlamaBoard、TensorBoard、Wandb、MLflow、SwanLab 等等。
极速推理：基于 vLLM 或 SGLang 的 OpenAI 风格 API、浏览器界面和命令行接口。

## 软硬件依赖

![](Image/2025-03-26-23-41-36.png) ![](Image/2025-03-26-23-41-58.png)

![](Image/2025-03-26-23-45-00.png)

## 安装

![](Image/2025-03-26-23-48-04.png)

git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
新建虚拟环境再执行下面的指令
cd LLaMA-Factory
pip install -e ".[torch,metrics]"

![](Image/2025-03-26-23-54-15.png)

conda create -n llamafactory python==3.10 -y

![](Image/2025-03-26-23-59-14.png)

激活虚拟环境

![](Image/2025-03-26-23-59-42.png)

可以先装基础环境，不装".[torch,metrics]"的东西

pip install -e .

![](Image/2025-03-27-00-02-57.png)

安装完成

![](Image/2025-03-27-00-14-48.png)

# 3.使用 LLaMA-Factory 微调 Qwen

## 更改助手名称
可以将identity.json下载到本地，然后将{{name}}替换为想要的名称重新上传

![](Image/2025-03-27-00-13-14.png)

## 启动服务

![](Image/2025-03-27-00-16-42.png)

llamafactory-cli webui

执行完成后会跳转到浏览器，注意是采用VScode链接远程服务器，如果换其他方法不一定可以

![](Image/2025-03-27-00-19-33.png)

## 配置数据集

打开dataset_info.json，

![](Image/2025-03-27-00-27-57.png)

![](Image/2025-03-27-00-33-37.png)

下面几个自动适应，一般不用改

![](Image/2025-03-27-00-35-34.png)

![](Image/2025-03-27-00-38-46.png)

选择数据集

![](Image/2025-03-27-00-42-43.png)

预览数据集确认是否正确

![](Image/2025-03-27-00-41-24.png)

训练轮数不能太小，至少要大于300

![](Image/2025-03-27-00-45-49.png)

截断长度即maxlength，给长了会占用显存，给的值小会减少显存占用，但是要保证满足回复长度的需求

![](Image/2025-03-27-00-50-21.png)

![](Image/2025-03-27-00-49-40.png)

批处理大小即批次，要根据显存大小来调

![](Image/2025-03-27-00-52-35.png)

“验证集比例”要设置一下，另外两个不用管

![](Image/2025-03-27-00-54-09.png)

一般“LoRA 缩放系数”是“LoRA 秩”的两倍，具体怎么调需要技巧，后面再说

![](Image/2025-03-27-00-55-44.png)

“设备数量”指的是有几张卡，“配置路径”指的是我们上面设置的参数保存位置

![](Image/2025-03-27-00-58-05.png)

点击开始

![](Image/2025-03-27-01-04-13.png)

如果出现RuntimeError:CUDA error:out of memory,要减小批次，或者检查是否有其他地方占用显存比较严重，比如说还开了其他模型

![](Image/2025-03-27-01-03-30.png)

Num examples是计算出来的样本数量

![](Image/2025-03-27-01-11-31.png)

批次大小要根据显存占用情况来调，如果显存占用比较低可以增大批次

![](Image/2025-03-27-01-17-43.png)

点击中断

![](Image/2025-03-27-01-18-30.png)

等待准备就绪

![](Image/2025-03-27-01-28-40.png)

修改批次大小重新开始，一直调到显存占用90%左右

![](Image/2025-03-27-01-20-12.png)

权重保存位置如下，默认是100个epoch保存一次权重,可以在浏览器中进行更改

![](Image/2025-03-27-01-37-06.png)

“日志间隔”是在终端每5个epoch输出一次，“保存间隔”是每100个epoch保存一次权重

![](Image/2025-03-27-01-39-28.png)

验证集和保存权重同步，所以保存间隔也控制验证

![](Image/2025-03-27-01-46-11.png)

## 继续训练的方法

首先，在界面中点击中断和开始就是继续训练

![](Image/2025-03-27-01-51-32.png)

但是如果出现界面崩掉的情况，继续训练就要手动设置，比如说我已经有了一个权重

![](Image/2025-03-27-01-53-14.png)


将权重设置到“检查点路径”中，然后开始就行

![](Image/2025-03-27-01-54-20.png)

## 训练完成验证

### 不加载权重的情况

不加载权重模型会按照以前的知识来介绍自己

![](Image/2025-03-27-02-02-32.png)


![](Image/2025-03-27-02-03-52.png)

## 加载权重后测试

![](Image/2025-03-27-02-07-18.png)